In [ ]:
from dataclasses import dataclass
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser

In [ ]:
config = ConfigParser()
config.read("../.config")

In [ ]:
API_KEY = config['GCP']['API_KEY']
radius = 1500  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [ ]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None


latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")


In [ ]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


In [ ]:
places['results'][0]

In [ ]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


In [ ]:
place_details['result']

In [ ]:
# Connect to DuckDB
con = duckdb.connect('places.db')
df = con.execute('SELECT * FROM places').fetchdf()

In [ ]:
con.close()

In [ ]:
def get_cities_towns(api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address=Luxembourg&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        results = data['results']
        places = []
        for result in results:
            places.append(result['formatted_address'])
        return places
    else:
        print(f"Error: {data['status']}")


cities_towns = get_cities_towns(API_KEY)
print(cities_towns)
